In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!ls "/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part1/Day3-FeatureSelection/notebooks"

In [ ]:
#ROOT_DIR = "/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part1/Day3-FeatureSelection/notebooks"
ROOT_DIR = "https://media.githubusercontent.com/media/yapay-ogrenme/casgem-eu-project-training-on-data-mining/9f260a425106bda386031bb6880a635c6d522c96/PART1/Day3-FeatureSelection/notebooks"
DATASET_PATH = ROOT_DIR + "/datasets/"

# Özellik Mühendisliği (Feature Engineering)

Amaç, verilerinizi eldeki soruna daha uygun hale getirmektir.

Isı indeksi ve rüzgar soğuğu gibi "görünür sıcaklık" ölçümlerini göz önünde bulundurun. Bu miktarlar, doğrudan ölçebileceğimiz şeyler olan hava sıcaklığı, nem ve rüzgar hızına dayalı olarak insanlara algılanan sıcaklığı ölçmeye çalışır. Görünür bir sıcaklığı bir tür özellik mühendisliğinin sonucu olarak düşünebilirsiniz, gözlemlenen verileri bizim gerçekten önemsediğimiz şeyle daha alakalı hale getirme girişimi: 

Özellik mühendisliğini şu amaçlarla gerçekleştirebilirsiniz:

* bir modelin tahmin performansını iyileştirmek
* hesaplama veya veri ihtiyaçlarını azaltmak
* sonuçların yorumlanabilirliğini geliştirmek


## Özellik Mühendisliğinin Yol Gösterici İlkesi

**Bir özelliğin kullanışlı olması için, modelinizin öğrenebildiği hedefle bir ilişkisi olmalıdır**. Örneğin, doğrusal modeller yalnızca doğrusal ilişkileri öğrenebilir. Bu nedenle, doğrusal bir model kullanırken amacınız, özellikleri hedefle olan ilişkilerini doğrusal hale getirmek için dönüştürmektir.

Buradaki ana fikir, bir özelliğe uyguladığınız dönüşümün özünde modelin kendisinin bir parçası haline gelmesidir. Diyelim ki bir tarafın "Uzunluğundan (Length)" kare arsaların "Fiyatını(Price)" tahmin etmeye çalışıyordunuz. Doğrusal bir modeli doğrudan "Uzunluk"a sığdırmak kötü sonuçlar verir: ilişki doğrusal değildir.

<figure style="padding: 1em;">
<img src="https://i.imgur.com/5D1z24N.png" width=300, alt="A scatterplot of Length along the x-axis and Price along the y-axis, the points increasing in a curve, with a poorly-fitting line superimposed.">
<figcaption style="textalign: center; font-style: italic"><center>Doğrusal bir model, özellik olarak yalnızca `Uzunluk` ile yetersiz uyum sağlar.
</center></figcaption>
</figure>

Ancak, 'Alan' elde etmek için 'Uzunluk' özelliğinin karesini alırsak, doğrusal bir ilişki yaratırız. Özellik kümesine "Alan" eklenmesi, bu doğrusal modelin artık bir parabole sığabileceği anlamına gelir. Diğer bir deyişle, bir özelliğin karesini almak, doğrusal modele karesi alınmış özellikleri sığdırma yeteneği verdi.

<figure style="padding: 1em;">
<img src="https://i.imgur.com/BLRsYOK.png" width=600, alt="Left: Area now on the x-axis. The points increasing in a linear shape, with a well-fitting line superimposed. Right: Length on the x-axis now. The points increase in a curve as before, and a well-fitting curve is superimposed.">
<figcaption style="textalign: center; font-style: italic"><center><strong>Sol:</strong> Alana uyum çok daha iyi. <strong>Sağ:</strong> Bu da Uzunluğa uyumu daha iyi hale getirir.
</center></figcaption>
</figure>

Bu size, özellik mühendisliğine yatırım yapıldığında neden bu kadar yüksek bir zaman getirisi olabileceğini göstermelidir. Modelinizin öğrenemediği ilişkiler ne olursa olsun, dönüşümler yoluyla kendinize sağlayabilirsiniz. Özellik setinizi geliştirirken, modelinizin en iyi performansı elde etmek için hangi bilgileri kullanabileceğini düşünün.

# Örnek - Beton Formülasyonları #

Bir veri kümesine birkaç sentetik özellik eklemenin, random forest modelinin tahmin performansını nasıl iyileştirebileceğini göreceğiz.

[*Concrete*](https://www.kaggle.com/sinamhd9/concrete-comprehensive-strength) veri kümesi, çeşitli beton formülasyonlarını ve ortaya çıkan ürünün, ne kadar yüke dayandığının bir ölçüsü olan *compressive strength(basınç dayanımını)* içerir. Bu bir tür beton dayanıklılığıdır. Bu veri setinin görevi, formülü verilen betonun basınç dayanımını tahmin etmektir.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv(DATASET_PATH+"concrete.csv")
df.head()

Burada, her bir beton çeşidine giren çeşitli malzemeleri görebilirsiniz. Bunlardan türetilen bazı ek sentetik özelliklerin eklenmesinin, bir modelin aralarındaki önemli ilişkileri öğrenmesine nasıl yardımcı olabileceğini birazdan göreceğiz.

İlk önce modeli artırılmamış veri kümesi üzerinde eğiterek bir temel oluşturacağız. Bu, yeni özelliklerimizin gerçekten yararlı olup olmadığını belirlememize yardımcı olacaktır.

Bunun gibi temeller oluşturmak, özellik mühendisliği sürecinin başlangıcında iyi bir uygulamadır. Temel puan, yeni özelliklerinizin korunmaya değer olup olmadığına veya bunları atıp muhtemelen başka bir şey denemeniz gerekip gerekmediğine karar vermenize yardımcı olabilir.

In [ ]:
X = df.copy()
y = X.pop("CompressiveStrength")

# Train and score baseline model
baseline = RandomForestRegressor(criterion="mae", random_state=0)
baseline_score = cross_val_score(
    baseline, X, y, cv=5, scoring="neg_mean_absolute_error"
)
baseline_score = -1 * baseline_score.mean()

print(f"MAE Baseline Score: {baseline_score:.4}")

Evde yemek pişirirseniz, bir tarifteki malzemelerin *oranının(ratio)* genellikle tarifin nasıl sonuçlanacağı konusunda mutlak miktarlarından daha iyi bir tahmin edici olduğunu biliyor olabilirsiniz. O halde yukarıdaki özelliklerin oranlarının 'Basınç Dayanımı(CompressiveStrength)' için iyi bir tahmin edici olacağını düşünebiliriz.

Veri kümesine üç yeni oran özellik ekleyelim

In [ ]:
X = df.copy()
y = X.pop("CompressiveStrength")

# Create synthetic features
X["FCRatio"] = X["FineAggregate"] / X["CoarseAggregate"]
X["AggCmtRatio"] = (X["CoarseAggregate"] + X["FineAggregate"]) / X["Cement"]
X["WtrCmtRatio"] = X["Water"] / X["Cement"]

X

In [ ]:
# Train and score model on dataset with additional ratio features
model = RandomForestRegressor(criterion="mae", random_state=0)
score = cross_val_score(
    model, X, y, cv=5, scoring="neg_mean_absolute_error"
)
score = -1 * score.mean()

print(f"MAE Score with Ratio Features: {score:.4}")

Ve elbette, performans arttı! Bu, bu yeni oran özelliklerinin, modele daha önce tespit etmediği önemli bilgileri gösterdiğinin kanıtıdır.
